In [ ]:
# Core analysis packages
import numpy as np
import os, sys
import pandas as pd
from scipy import stats
from scipy.special import comb
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Plotting packages
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
# Load file (from same directory as the notebook)
df = pd.read_excel("data.xlsx", index_col="Response ID")
df.head()

,GENDER,AGE,PARTY,TWITTER,TRUST,RU1,RU2,RU3,RU4,RU5,...,Post23,Post24,Post25,Post26,Post27,Post28,Post29,Post30,Post31,Post32
Response ID,,,,,,,,,,,,,,,,,,,,,
R_0cj5dsJg2wfpiuJ,1,18,1,0,95,4.0,26,0,-5,4,...,69,60,20,58,84,22,42,77,90,71
R_0rkhLjwWPHHjnTX,0,19,2,1,76,-5.0,16,3,-1,7,...,58,82,38,61,36,40,62,68,46,43
R_10BMNpjhInMfUeO,1,18,1,1,86,-5.0,-2,5,5,17,...,35,46,39,65,44,42,53,55,45,35
R_120iGR6WlLnbZnI,0,22,1,0,95,23.0,-10,-40,22,6,...,14,76,20,61,87,82,63,19,97,37
R_12qW8cDY0bNlId2,0,19,3,0,76,18.0,-12,1,16,11,...,17,81,31,83,82,76,43,33,82,47


## Reshaping data

COMMENTS HERE:

In [ ]:
data = pd.wide_to_long(df.reset_index(), stubnames=["RU", "Pre", "Post"], i="Response ID", j="Item ID").reset_index()
data

,Response ID,Item ID,TWITTER,AGE,GENDER,PARTY,TRUST,RU,Pre,Post
0,R_0cj5dsJg2wfpiuJ,1,0,18,1,1,95,4.0,83,87.0
1,R_0rkhLjwWPHHjnTX,1,1,19,0,2,76,-5.0,66,61.0
2,R_10BMNpjhInMfUeO,1,1,18,1,1,86,-5.0,67,62.0
3,R_120iGR6WlLnbZnI,1,0,22,0,1,95,23.0,74,97.0
4,R_12qW8cDY0bNlId2,1,0,19,0,3,76,18.0,81,99.0
...,...,...,...,...,...,...,...,...,...,...
6395,R_xapQxguTwA3Juh3,32,0,18,1,1,76,36.0,63,27.0
6396,R_XMS13V10vkvYag9,32,0,18,1,3,76,13.0,48,35.0
6397,R_ykkxJ7f40bzTEaZ,32,0,19,1,1,89,9.0,84,75.0
6398,R_ZDXFN47SOcbCJpv,32,0,21,0,2,100,38.0,62,24.0


DataFrames can be saved with `DataFrame.to_csv()` or `DataFrame.to_excel()`. If you're doing your analysis in python anyways, csvs are much easier to manage:

In [ ]:
# data.to_csv("longdata.csv")
# data.to_excel("longdata.xlsx")

`pandas.wide_to_long()` is the best-case resource for converting wide-format (spreadsheet) data into long-format data for analysis, but sometimes isn't applicable. In those cases, `pandas.melt()` is the way.

For these data, `pandas.melt()` can't transform all three data variables at once (`RU`, `Pre`, and `Post`), but it can do any _one_ of them. So let's restrict our dataframe to just the `RU` variable and show how that works in contrast:

In [ ]:
subdf = df.loc[:, "Response ID":"RU32"]
subdf.head()

,Response ID,GENDER,AGE,PARTY,TWITTER,TRUST,RU1,RU2,RU3,RU4,...,RU23,RU24,RU25,RU26,RU27,RU28,RU29,RU30,RU31,RU32
0,R_0cj5dsJg2wfpiuJ,1,18,1,0,95,4.0,26,0,-5,...,-3,-1,1,5,-18,39,18,-14,-1,-36
1,R_0rkhLjwWPHHjnTX,0,19,2,1,76,-5.0,16,3,-1,...,2,-11,5,4,8,-3,0,-5,11,0
2,R_10BMNpjhInMfUeO,1,18,1,1,86,-5.0,-2,5,5,...,43,-1,0,15,-1,20,8,0,16,10
3,R_120iGR6WlLnbZnI,0,22,1,0,95,23.0,-10,-40,22,...,-5,-4,-2,-37,-15,-1,-33,40,-28,-18
4,R_12qW8cDY0bNlId2,0,19,3,0,76,18.0,-12,1,16,...,13,-6,-11,-15,-10,-40,20,19,6,16


In [ ]:
long = pd.melt(
    subdf, 
    id_vars=["Response ID", "GENDER", "AGE", "PARTY", "TWITTER", "TRUST"], 
    value_vars=[f"RU{i+1}" for i in range(32)], var_name="Item ID", value_name='RU'
)
long

,Response ID,GENDER,AGE,PARTY,TWITTER,TRUST,Item ID,RU
0,R_0cj5dsJg2wfpiuJ,1,18,1,0,95,RU1,4.0
1,R_0rkhLjwWPHHjnTX,0,19,2,1,76,RU1,-5.0
2,R_10BMNpjhInMfUeO,1,18,1,1,86,RU1,-5.0
3,R_120iGR6WlLnbZnI,0,22,1,0,95,RU1,23.0
4,R_12qW8cDY0bNlId2,0,19,3,0,76,RU1,18.0
...,...,...,...,...,...,...,...,...
6395,R_xapQxguTwA3Juh3,1,18,1,0,76,RU32,36.0
6396,R_XMS13V10vkvYag9,1,18,3,0,76,RU32,13.0
6397,R_ykkxJ7f40bzTEaZ,1,19,1,0,89,RU32,9.0
6398,R_ZDXFN47SOcbCJpv,0,21,2,0,100,RU32,38.0


The opposite of "melting" a wide-format data table into a long-format data table is "pivoting". We can undo the melt operation above:

In [ ]:
wide = pd.pivot(
    long,
    index=["Response ID", "GENDER", "AGE", "PARTY", "TWITTER", "TRUST"],
    columns="Item ID",
    values="RU"
).reset_index()
wide

Item ID,Response ID,GENDER,AGE,PARTY,TWITTER,TRUST,RU1,RU10,RU11,RU12,...,RU3,RU30,RU31,RU32,RU4,RU5,RU6,RU7,RU8,RU9
0,R_0cj5dsJg2wfpiuJ,1,18,1,0,95,4.0,6.0,6.0,15.0,...,0.0,-14.0,-1.0,-36.0,-5.0,4.0,-31.0,-1.0,22.0,4.0
1,R_0rkhLjwWPHHjnTX,0,19,2,1,76,-5.0,-5.0,5.0,-2.0,...,3.0,-5.0,11.0,0.0,-1.0,7.0,-5.0,3.0,-2.0,-8.0
2,R_10BMNpjhInMfUeO,1,18,1,1,86,-5.0,-3.0,-1.0,14.0,...,5.0,0.0,16.0,10.0,5.0,17.0,2.0,-4.0,4.0,12.0
3,R_120iGR6WlLnbZnI,0,22,1,0,95,23.0,-15.0,3.0,0.0,...,-40.0,40.0,-28.0,-18.0,22.0,6.0,-10.0,2.0,-41.0,11.0
4,R_12qW8cDY0bNlId2,0,19,3,0,76,18.0,22.0,-6.0,13.0,...,1.0,19.0,6.0,16.0,16.0,11.0,-1.0,16.0,12.0,-4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,R_vZuFb6GDAWb2dhL,0,20,1,1,90,-4.0,-7.0,-11.0,-9.0,...,5.0,-5.0,-12.0,11.0,25.0,-3.0,15.0,-6.0,-4.0,26.0
196,R_wTYaq1bnIPk7pIJ,1,19,3,0,90,-3.0,-30.0,8.0,-17.0,...,1.0,-17.0,7.0,-12.0,8.0,-10.0,0.0,16.0,20.0,-5.0
197,R_wZ3U1Md90MfZpa9,1,19,1,1,90,6.0,0.0,6.0,2.0,...,31.0,-1.0,6.0,-29.0,1.0,-5.0,2.0,0.0,-14.0,-4.0
198,R_xapQxguTwA3Juh3,1,18,1,0,76,-13.0,5.0,-17.0,4.0,...,3.0,49.0,-23.0,36.0,-32.0,-22.0,3.0,2.0,-2.0,11.0


While you're less likely to need `pandas.pivot()`, its counterparts `pandas.pivot_table()` and `pandas.crosstab()` work similarly, and can be very useful for generating _summary statistics_ over your data.

Let's compute the mean `RU` over different groups of participants and items in our dataset:

In [ ]:
pd.pivot_table(
    data, 
    index=["GENDER", "PARTY"],
    # columns=["Item ID"], # Uncomment this to decompose by Item ID as well
    values=["RU", "Pre", "Post"],
    aggfunc="mean", # Can change this to "std", "var", "median", etc.
)

Post        Pre        RU
GENDER PARTY                                
0      1      50.398026  51.418586  2.186678
       2      51.673611  54.961806  1.947917
       3      48.421875  50.180990  1.048177
1      1      49.359793  49.883427  3.081001
       2      52.669271  53.421875  2.070312
       3      51.319196  52.535714  2.073661

This can be replicated with `DataFrame.groupby()`:

In [ ]:
data.groupby(["GENDER", "PARTY"]).agg("mean")

Item ID   TWITTER        AGE      TRUST        RU        Pre  \
GENDER PARTY                                                                 
0      1         16.5  0.236842  20.289474  88.578947  2.186678  51.418586   
       2         16.5  0.222222  20.333333  80.444444  1.947917  54.961806   
       3         16.5  0.291667  19.833333  85.666667  1.048177  50.180990   
1      1         16.5  0.235955  19.123596  88.595506  3.081001  49.883427   
       2         16.5  0.333333  18.833333  78.750000  2.070312  53.421875   
       3         16.5  0.214286  19.321429  80.107143  2.073661  52.535714   

                   Post  
GENDER PARTY             
0      1      50.398026  
       2      51.673611  
       3      48.421875  
1      1      49.359793  
       2      52.669271  
       3      51.319196

## Subsampling data

Suppose you wanted to analyze a particular subset of your sample, e.g. only the female participants. We can use the `DataFrame.loc[..., ...]` method to do this. The first argument to `loc` refers to the indices/rows, and the second argument to the columns. By default, a colon `:` means "all of them".

The following _logical operation_ flags, for each participant, whether they are female (`== 0`, `True`) or not:

In [ ]:
df.GENDER==0 
# df.loc[:, "GENDER"]==0 also works

Response ID
R_0cj5dsJg2wfpiuJ    False
R_0rkhLjwWPHHjnTX     True
R_10BMNpjhInMfUeO    False
R_120iGR6WlLnbZnI     True
R_12qW8cDY0bNlId2     True
                     ...  
R_xapQxguTwA3Juh3    False
R_XMS13V10vkvYag9    False
R_ykkxJ7f40bzTEaZ    False
R_ZDXFN47SOcbCJpv     True
R_ZpYHWVd91u6fjBT     True
Name: GENDER, Length: 200, dtype: bool

`DataFrame.loc[df.GENDER==0, :]` keeps every `True` row and discards every `False` row, and all columns, marked by the colon `:`.

In [ ]:
df.loc[df.GENDER==0, :].head()

,GENDER,AGE,PARTY,TWITTER,TRUST,RU1,RU2,RU3,RU4,RU5,...,Post23,Post24,Post25,Post26,Post27,Post28,Post29,Post30,Post31,Post32
Response ID,,,,,,,,,,,,,,,,,,,,,
R_0rkhLjwWPHHjnTX,0,19,2,1,76,-5.0,16,3,-1,7,...,58,82,38,61,36,40,62,68,46,43
R_120iGR6WlLnbZnI,0,22,1,0,95,23.0,-10,-40,22,6,...,14,76,20,61,87,82,63,19,97,37
R_12qW8cDY0bNlId2,0,19,3,0,76,18.0,-12,1,16,11,...,17,81,31,83,82,76,43,33,82,47
R_1C9L7MAYTX0RKuA,0,21,3,0,100,-4.0,-15,-1,-10,8,...,38,39,30,59,21,71,40,26,69,32
R_1dGPEFV0s6vdlAR,0,21,2,0,56,1.0,2,-5,32,-3,...,13,22,16,78,56,26,58,61,62,55


_A lot can be done with this_. Let's enumerate a few:

In [ ]:
# Check the age of all female participants
df.loc[df.GENDER==0, "AGE"]
# Check whether female participants WITH LESS-THAN-MEDIAN TRUST have Twitter
df.loc[np.logical_and(df.GENDER==0, df.TRUST<df.TRUST.median()), "TWITTER"]
# That would return a pd.Series by default; adding brackets returns a DataFrame:
df.loc[np.logical_and(df.GENDER==0, df.TRUST<df.TRUST.median()), ["TWITTER"]]

# Get the RU data vars for Democratic participants
df.loc[df.PARTY==1, "RU1":"RU32"] # This colon notation grabs an ordered range


,RU1,RU2,RU3,RU4,RU5,RU6,RU7,RU8,RU9,RU10,...,RU23,RU24,RU25,RU26,RU27,RU28,RU29,RU30,RU31,RU32
Response ID,,,,,,,,,,,,,,,,,,,,,
R_0cj5dsJg2wfpiuJ,4.0,26,0,-5,4,-31,-1,22,4,6,...,-3,-1,1,5,-18,39,18,-14,-1,-36
R_10BMNpjhInMfUeO,-5.0,-2,5,5,17,2,-4,4,12,-3,...,43,-1,0,15,-1,20,8,0,16,10
R_120iGR6WlLnbZnI,23.0,-10,-40,22,6,-10,2,-41,11,-15,...,-5,-4,-2,-37,-15,-1,-33,40,-28,-18
R_1Cmg1E3Fn2Evh4J,12.0,12,27,5,-4,1,23,11,1,15,...,-5,-3,-10,5,2,13,0,3,20,-10
R_1etaEYMiW0ydSuF,-1.0,-2,-12,21,11,-26,-1,-13,-1,-14,...,10,5,12,-8,-4,11,-5,-6,5,-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R_WrsI8uTJ4myCmqd,2.0,-6,54,-13,-1,-31,-19,-17,-80,54,...,26,49,0,80,-4,0,0,-23,5,4
R_wZ3U1Md90MfZpa9,6.0,2,31,1,-5,2,0,-14,-4,0,...,0,0,5,9,3,24,-8,-1,6,-29
R_xapQxguTwA3Juh3,-13.0,3,3,-32,-22,3,2,-2,11,5,...,-19,-12,2,8,-36,-24,-33,49,-23,36
